<a href="https://colab.research.google.com/github/WillianXG/trabalho_pao/blob/main/Desafio_do_pao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'dados-lanchonete-pao:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5219122%2F8701963%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240617%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240617T015722Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D54a5716ed6636fe13c2c52a89978c066721510beae900baf3069dfde374ee69ae5bbb37098145afcfe9d1bbce2a1d9395f7e03530ea27dfe81fcc8e7ee6c5505b36f36a3ef216b10d01ba89ba0c3c47c130a8f1d455dee37b316430224c090bcc088c01576a6a32b23bb71a02d3ed4e2352c9013b1c84649d88db885526f6ca3778196022ea7b3df5e67ecfc7bf58cb2ccc59653a21f87cdb78e5c69ede72a1558799453bd697fe77d95281ab67fd7b3ec1382d0002c20a234fc877ebe7dce483ddf54f3103ac22a5bf2976c3dd7735e941d01265fd2fdaffc5e8447a4364512f3f82414169f4b5896687e009e536db97946ec07e0255bd0e91b37543928c8c2'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 20279 bytes downloaded
Downloaded and uncompressed: dados-lanchonete-pao
Data source import complete.


In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/dados-lanchonete-pao/DataBase po.xlsx


# Desafio do pão
## Criar um CSV resultante quem tem a maior probabilidade de comprar pão durante a semana

Deve ser feito uma analise do DataFrame e criar como resultante um CSV para mostar quais as pessoas com maior possibilidade de comprar pão durante a semana, e o CSV deve conter as seguintes colunas:

1. Nome da Pessoa
2. Valor da possibilidade de compra por semana
3. Valor Minimo de compra de pão por dia
4. Valor Maximo de compra de pão por dia

Com isso os 3 primeiros são as pessoas com maior probabildiade de compra de pão durante a semana.

Para chegar nessa conclusão pode utilziar cruzamento de valores como **Se a pessoa Trabalha e qual foi o Valor da sua ultima compra** , e assim verificar quais pessoas tem a probabilidade de comprar um pão durante a semana.

Alem disso o codigo não deve ter comentarios descritivos para explicar o que o codigo faz. E o DataFrame deve ser tratado tanto com valores ausentes e/ou valores incorretos, relizando a limpeza de dados.

**Dica** : Crie um (ou mais) sistema de classificação, assim utilize essas constantes para avaliar pessoa por pessoa se ela poderá ser uma compradora do pão durante a semana ou não.

In [ ]:
#Tratamento de dados para o DataFrame original
Dados_pao = pd.read_excel('/kaggle/input/dados-lanchonete-pao/DataBase po.xlsx')

def trat_int(df):
  try:
    valor_int = int(df)
    if valor_int != int:
      return int(df)
    else:
      return df
  except ValueError:
    return df


def trat(df):
    try:
        valor_int = int(df)
        if valor_int == 2:
            return 'Baixa'
        elif valor_int == 5:
            return 'Média'
        elif valor_int == 12:
            return 'Alta'
        else:
            return df
    except ValueError:
        return df

def Sex(df):
  try:
    if df == 'masculino':
      return 'M'
    elif df == 'feminino':
      return 'F'
    else:
      return df
  except ValueError:
    return df

def trab(df):
    for i, row in df.iterrows():
        if row['Profissão'] == 'Não informado':
            df.at[i, 'Trabalha'] = 'Não'
        elif row['Profissão'] == 'Estudante':
            df.at[i, 'Trabalha'] = 'Não'
        else:
            df.at[i, 'Trabalha'] = 'Sim'
    return df

mediana_idade = Dados_pao['idade'].median()
mediana_peso = Dados_pao['Peso'].median()


Dados_pao['Sexo'] = Dados_pao['Sexo'].apply(Sex)
Dados_pao['Interação social'] = Dados_pao['Interação social'].apply(trat)
Dados_pao['Profissão'] = Dados_pao['Profissão'].fillna('Não informado')
Dados_pao['Comida Favorita'] = Dados_pao['Comida Favorita'].fillna('Não informado')
Dados_pao['idade'] = Dados_pao['idade'].fillna(mediana_idade)
Dados_pao['pão comprados'] = Dados_pao['pão comprados'].fillna(0)
Dados_pao['numero de amigos'] = Dados_pao['numero de amigos'].fillna(0)
Dados_pao['Valor gasto por semana'] = Dados_pao['Valor gasto por semana'].fillna(0)
Dados_pao['Peso'] = Dados_pao['Peso'].fillna(mediana_peso)
Dados_pao['Estado civil'] = Dados_pao['Estado civil'].fillna('Não informado')
Dados_pao['Trabalha'] = None
Dados_pao['idade'] = Dados_pao['idade'].apply(trat_int)
Dados_pao['pão comprados'] = Dados_pao['pão comprados'].apply(trat_int)
Dados_pao['numero de amigos'] = Dados_pao['numero de amigos'].apply(trat_int)
Dados_pao = trab(Dados_pao)

df = Dados_pao

df

,id,pessoa,idade,Sexo,pão comprados,Frequencia,Faltas,altura,Peso,Trabalha,Profissão,Faculdade,Estado civil,numero de amigos,valor da ultima compra,Fumante,Bebida alcolica,Valor gasto por semana,Comida Favorita,Interação social
0,1,João Silva,25,M,5,Semanal,2,175,70.0,Sim,Engenheiro,USP,Solteiro,10,5,Não,Sim,20.0,Pizza,Alta
1,2,Maria Oliveira,30,M,3,Quinzenal,0,165,60.0,Sim,Professora,UNICAMP,Casada,15,1,Não,Não,15.0,Lasanha,Média
2,3,Carlos Santos,28,M,4,Mensal,1,180,80.0,Não,Não informado,PUC,Divorciado,8,7,Sim,Sim,25.0,Sushi,Alta
3,4,Ana Costa,22,F,6,Semanal,3,170,65.0,Não,Estudante,USP,Solteira,20,3,Não,Sim,10.0,Hamburguer,Alta
4,5,Pedro Almeida,28,M,2,Mensal,0,178,69.5,Sim,Médico,UNIFESP,Casado,0,2,Não,Não,30.0,Não informado,Alta
5,6,Julia Ferreira,27,F,5,Semanal,2,160,55.0,Sim,Designer,FAAP,Solteira,25,6,Não,Sim,0.0,Salada,Média
6,7,Marcos Pereira,33,M,3,Quinzenal,0,182,90.0,Não,Não informado,FGV,Casado,10,1,Sim,Não,18.0,Pizza,Alta
7,8,Beatriz Lima,26,F,4,Mensal,1,168,58.0,Sim,Jornalista,USP,Solteira,18,8,Não,Sim,19.0,Massa,Média
8,9,Ricardo Sousa,31,M,0,Quinzenal,1,175,75.0,Sim,Contador,Mackenzie,Casado,12,1,Não,Não,23.0,Feijoada,Alta
9,10,Larissa Gomes,24,F,5,Semanal,3,166,62.0,Sim,Publicitária,ESPM,Solteira,22,4,Não,Sim,15.0,Sushi,Alta


In [ ]:
#Tratamento de dados para o DataFrame original
data = {
        'Nome': [],
        'Probabilidade': [],
        'Minimo': [],
        'Maximo': []
  }

def probpao(df):


    for i, row in df.iterrows():
        if row['Trabalha'] == 'Sim' and row['Frequencia'] == 'Semanal' and row['Valor gasto por semana'] >= 20 and row['Faltas'] < 5:
            data['Nome'].append(row['pessoa'])
            data['Probabilidade'].append('Alta')
            data['Minimo'].append(1)
            data['Maximo'].append(1)
        elif row['Trabalha'] == 'Sim' and row['Valor gasto por semana']  <=15 and row['valor da ultima compra'] <=9 or row['Frequencia'] == 'Quinzenal' or row['Frequencia'] == 'Mensal':
            data['Nome'].append(row['pessoa'])
            data['Probabilidade'].append('Média')
            data['Minimo'].append(2)
            data['Maximo'].append(2)
        else:
            data['Nome'].append(row['pessoa'])
            data['Probabilidade'].append('Baixa')
            data['Minimo'].append(0)
            data['Maximo'].append(0)



probpao(df)


data = pd.DataFrame(data)
data




,Nome,Probabilidade,Minimo,Maximo
0,João Silva,Alta,1,1
1,Maria Oliveira,Média,2,2
2,Carlos Santos,Média,2,2
3,Ana Costa,Baixa,0,0
4,Pedro Almeida,Média,2,2
5,Julia Ferreira,Média,2,2
6,Marcos Pereira,Média,2,2
7,Beatriz Lima,Média,2,2
8,Ricardo Sousa,Média,2,2
9,Larissa Gomes,Média,2,2


In [ ]:
# #Resultado em formato CSV
# Resultado_pao  =  Dados_pao_tratados.to_csv("Probabilidade_pao.csv")


# def dividir_em_partes(total, divisores):
#     # Calcula a soma dos divisores
#     soma_divisores = sum(divisores)

#     # Inicializa a lista para armazenar as partes divididas
#     partes = []

#     # Calcula cada parte proporcional
#     for divisor in divisores:
#         parte = total * (divisor / soma_divisores)
#         partes.append(parte)

#     return partes

# # Exemplo de uso
# total = 100
# divisores = [1, 2, 3]

# partes_divididas = dividir_em_partes(total, divisores)
# print(partes_divididas)  # Saída: [16.666666666666668, 33.333333333333336, 50.0]
# print(sum(partes_divididas))  # Saída: 100.0 (confirma que a soma das partes é igual ao total)


def contagem_da_semana(dias, faltas):
  if faltas >= len(dias):
      return []
  elif faltas == 0:
    return dias
  else:
      return dias[:-faltas]

def calc_faltas(df):
    dias = [1, 2, 3, 4, 5]
    for i, row in df.iterrows():
        faltas = row['Faltas']
        if faltas >= 0:
            divisores = contagem_da_semana(dias,faltas)
            return divisores
        else:
            return('erro')
def calc_valores(df):
  for i, row in df.iterrows():


calc_valores(df)

[1, 2, 3]
0
[1, 2, 3, 4]
[1, 2]
0
[1, 2, 3]
0
[1, 2, 3, 4]
[1, 2, 3, 4]
[1, 2]
0
0
[1, 2, 3]
[1, 2, 3, 4]
[1, 2, 3, 4]
0
0
[1, 2, 3]
[1, 2, 3, 4]
[1, 2]
[1, 2, 3, 4]
[1, 2, 3]
[1, 2, 3, 4]
[1, 2]
0
[1, 2, 3]
0
[1, 2, 3, 4]
[1, 2, 3, 4]
[1, 2]
0
0
[1, 2, 3]
[1, 2, 3, 4]
[1, 2, 3, 4]
0
0
[1, 2, 3]
[1, 2, 3, 4]
[1, 2]
[1, 2, 3, 4]
[1, 2, 3]
[1, 2, 3, 4]
[1, 2]
0
[1, 2, 3]
0
[1, 2, 3, 4]
[1, 2, 3, 4]
[1, 2]
